<a href="https://colab.research.google.com/github/Kent-mak/Anti-Hate-dashboard/blob/main/Civil%20Regression%20BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch datasets transformers peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 29.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-man

Define Model with LoRA for regression

In [2]:
import torch
if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')
print("Using {}.".format(device_name))

Using cuda.


In [3]:

import torch.nn as nn


class RegressionModel(nn.Module):
    def __init__(self, base_model, num_labels=7):
        super(RegressionModel, self).__init__()
        self.model = base_model
        self.dropout = nn.Dropout(0.1)
        self.regressor = nn.Linear(self.model.config.hidden_size, num_labels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.model(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[0][:, 0]
        pooled_output = self.dropout(pooled_output)
        logits = self.regressor(pooled_output)
        logits = self.sigmoid(logits)

        if labels is not None:
            loss_fct = nn.MSELoss()
            loss = loss_fct(logits, labels)
            return loss, logits
        else:
            return logits


In [22]:
from peft import LoraConfig, get_peft_model
from transformers import AutoModel

model_name = "distilbert-base-uncased"
base_model = AutoModel.from_pretrained(model_name)

lora_config = LoraConfig(
    r=4,                      # Rank of the low-rank matrices
    lora_alpha=32,            # Scaling factor
    lora_dropout=0.1,         # Dropout probability for LoRA layers
    target_modules=["attention.q_lin", "attention.k_lin", "attention.v_lin"]  # Modules to which LoRA will be applied
)

model_with_lora = get_peft_model(base_model, lora_config)


In [23]:
# Load the tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Integrate LoRA with the custom regression model
model = RegressionModel(model_with_lora)

Work with the dataset

In [24]:
from datasets import load_dataset

columns_to_check = ['toxicity', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']

# Define a filter function
def filter_all_zeros(example):
    return any(example[col] != 0 for col in columns_to_check)

percentage = '[:10%]'

# dataset = load_dataset('google/civil_comments')
training_set = load_dataset('google/civil_comments', split= f'train{percentage}')
validation_set = load_dataset('google/civil_comments', split= f'validation{percentage}')
test_set = load_dataset('google/civil_comments', split= f'test[:10%]')

# filtered_training_set = training_set.filter(filter_all_zeros)
# filtered_validation_set = validation_set.filter(filter_all_zeros)


print(training_set)
print(validation_set)

Dataset({
    features: ['text', 'toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit'],
    num_rows: 180487
})
Dataset({
    features: ['text', 'toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit'],
    num_rows: 9732
})


In [25]:
# Define the preprocessing function
def preprocess_function(examples):
    inputs = tokenizer(examples['text'], truncation=True, padding='max_length')
    labels = torch.tensor(list(zip(
        examples['toxicity'],
        examples['severe_toxicity'],
        examples['obscene'],
        examples['threat'],
        examples['insult'],
        examples['identity_attack'],
        examples['sexual_explicit']
    )), dtype=torch.float32)

    inputs["labels"] = labels
    return inputs

# Apply the preprocessing function to each split
# encoded_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)
encoded_training_set = training_set.map(preprocess_function, batched=True, remove_columns=training_set.column_names)
encoded_validation_set = validation_set.map(preprocess_function, batched=True, remove_columns=validation_set.column_names)
encoded_test_set = test_set.map(preprocess_function, batched=True, remove_columns=test_set.column_names)
print(encoded_training_set)



Map:   0%|          | 0/9732 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 180487
})


Train the model

In [26]:
from transformers import Trainer, TrainingArguments

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
      # print(inputs)
      labels = inputs.get("labels").to(device_name)
      model.to(device_name)
      loss, logits = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=labels)
      return (loss, logits) if return_outputs else loss


In [27]:


# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=encoded_training_set,
    eval_dataset=encoded_validation_set,
)

# Fine-tune the model
trainer.train()
test_results = trainer.evaluate(encoded_test_set)
print(f"Test Results: {test_results}")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.007000,0.006763
2,0.006300,0.006184
3,0.006000,0.005997


Test Results: {'eval_loss': 0.006053806282579899, 'eval_runtime': 48.697, 'eval_samples_per_second': 199.848, 'eval_steps_per_second': 6.263, 'epoch': 3.0}


In [28]:
baseRegressionModel = RegressionModel(base_model)

training_args = TrainingArguments(
    output_dir="./base_results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
base_trainer = CustomTrainer(
    model=baseRegressionModel,
    args=training_args
)

base_test_results = base_trainer.evaluate(encoded_test_set)
print(f"Test Results: {base_test_results}")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Test Results: {'eval_loss': 0.2226308137178421, 'eval_runtime': 48.7219, 'eval_samples_per_second': 199.746, 'eval_steps_per_second': 6.26}


In [29]:
torch.save(model.state_dict(), "./model10%.pt")

In [30]:
from transformers import AutoTokenizer

loaded_model = RegressionModel(model_with_lora)
loaded_model.to(device_name)
loaded_model.load_state_dict(torch.load("./model5%.pt"))
loaded_model.eval()

new_tokenizer = AutoTokenizer.from_pretrained(model_name)
example_text = 'haha you guys are a bunch of losers.'
input = new_tokenizer(example_text, return_tensors="pt").to(device_name)

with torch.no_grad():
  logits = loaded_model(**input)

print(logits)
# print(loss)

FileNotFoundError: [Errno 2] No such file or directory: './model5%.pt'